# Modelling

In [0]:
# we need pillow version of 5.3.0
# we will uninstall the older version first
!pip uninstall -y Pillow
# install the new one
!pip install Pillow==5.3.0
# import the new one
import PIL
print(PIL.PILLOW_VERSION)
# this should print 5.3.0. If it doesn't, then restart your runtime:
# Menu > Runtime > Restart Runtime

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
cd '/content/drive/My Drive/Neural Style Transfer for Videos/ZFH'

In [0]:
# google colab does not come with torch installed

from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

# !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
# we will verify that GPU is enabled for this notebook
# following should print: CUDA is available!  Training on GPU ...
# 
# if it prints otherwise, then you need to enable GPU: 
# from Menu > Runtime > Change Runtime Type > Hardware Accelerator > GPU

import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [0]:
import PIL
print(PIL.PILLOW_VERSION)

In [0]:
# Import the necessary packages:
import cv2
import glob
from PIL import Image  # Python Image Library for image processing
import matplotlib.pyplot as plt  # Plotting
import numpy as np  # Numerical computation
import torch  # Neural network computation
from torch import optim  # optimizer to minimize the loss function
from torchvision import transforms, models  # Transformations on images and pre-trained models
import os, os.path  # To count the number of image files

In [0]:
# LOAD VGG19 (features only):
# vgg19.features 	: It consists of all the convolutional and pooling layers
# vgg19.classifier 	: It consists of the 3 linear classifier layers at the end

# We load in the pre-trained model and freeze the weights:
styleTransferModel = models.vgg19(pretrained=True).features

# Freeze all the VGG parameters since we are only optimizing the target image:
for params in styleTransferModel.parameters():
	params.requires_grad_(False)

In [0]:
# Check if GPU is available:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on GPU")
else:
    device = torch.device("cpu")
    print("Running on CPU")

# Move the model to the GPU if available:
styleTransferModel.to(device)

In [0]:
# Load the content and style images:
def load_image(image_path, max_size=480, shape=None):
    image = Image.open(image_path).convert('RGB')

    # Resize the image as a large image will slow down processing:
    if max(image.size) > max_size:
        img_size = max_size
    else:
        img_size = max(image.size)

    if shape is not None:
        img_size = shape

    img_transform = transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406),
                             (0.229, 0.224, 0.225))])
    # Discard the transparent alpha channel (that's the :3) and add the batch dimension:
    image = img_transform(image)[:3, :, :].unsqueeze(0)

    return image

In [0]:
# CONTENT AND STYLE FEATURES:
# Map the layer names to the names given in the paper:
def get_features(image, model, layers=None):
    if layers is None:
        layers = {'0': 'conv1_1',
                  '5': 'conv2_1',
                  '10': 'conv3_1',
                  '19': 'conv4_1',
                  '21': 'conv4_2',  # Content representation
                  '28': 'conv5_1',}

    features = {}
    x = image
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[layers[name]] = x

    return features

In [0]:
# GRAM MATRIX:
def gram_matrix(tensor):
  batch_size, depth, height, width = tensor.size()
  # Vectorize the input image tensor and add all the feature maps:
  tensor = tensor.view(depth, height * width)
  # Transpose the image tensor:
  tensor_t = tensor.t()
  # Compute the gram matrix by multiplying the matrix by its transpose:
  gram = torch.mm(tensor, tensor_t)

  return gram

In [0]:
def convert_video_to_frames(video_directory):
    # Load the input video:
    capture_video = cv2.VideoCapture(video_directory)
    # Read the input frame:
    success, frame = capture_video.read()
    # Set counter for number of frames read:
    count = 1

    while success:
        # Save the frame that is read:
        cv2.imwrite('input_content_frames/frame_%d.jpg' % count, frame)
        # Read the next input frame:
        success, frame = capture_video.read()
        print("Read a new frame: ", count, "Success: ", success)
        count += 1

In [0]:
# Split the video into frames:
convert_video_to_frames('input_video/IMG_6039.MOV')

In [0]:
# Function to un-normalize an image and convert from a Tensor image to a NumPy image for display or writing to disk:
def img_convert(tensor):
    # Display a tensor as an image:

    image = tensor.to("cpu").clone().detach()
    image = image.numpy().squeeze()
    image = image.transpose(1, 2, 0)
    image = image * np.array((0.229, 0.224, 0.225)) + np.array((0.485, 0.456, 0.406))
    image = image.clip(0, 1)

    return image

In [0]:
!git clone https://github.com/pathak22/pyflow.git

In [0]:
cd pyflow/

In [0]:
!python setup.py build_ext -i

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
# from __future__ import unicode_literals
import time
import argparse
import pyflow

def optical_flow(Image1, Image2, Image1_index, Image2_index):
  im1 = np.array(Image1, order='C').astype(float)
  im2 = np.array(Image2, order='C').astype(float)

  # Flow Options:
  alpha = 0.012
  ratio = 0.75
  minWidth = 20
  nOuterFPIterations = 7
  nInnerFPIterations = 1
  nSORIterations = 30
  colType = 0  # 0 or default:RGB, 1:GRAY (but pass gray image with shape (h,w,1))

  s = time.time()
  u, v, im2W = pyflow.coarse2fine_flow(
      im1, im2, alpha, ratio, minWidth, nOuterFPIterations, nInnerFPIterations,
      nSORIterations, colType)
  e = time.time()

  flow = np.concatenate((u[..., None], v[..., None]), axis=2)
  np.save('../flow/flow_matrix/outFlow_%s_%s.npy' %(Image1_index, Image2_index), flow)

  hsv = np.zeros(im1.shape, dtype=np.uint8)
  hsv[:, :, 0] = 255
  hsv[:, :, 1] = 255
  mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
  hsv[..., 0] = ang * 180 / np.pi / 2
  hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
  rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
  cv2.imwrite('../flow/outFlow_%s_%s.png' %(Image1_index, Image2_index), rgb)
  im_warped = im2W[:, :, ::-1] * 255
  cv2.imwrite('../flow/Warped_%s_%s.jpg' %(Image1_index, Image2_index), im_warped)

  return flow, im_warped

In [0]:
input_frames_length = len([name for name in os.listdir('../input_content_frames') if os.path.isfile(os.path.join('../input_content_frames', name))])
frame_interval = (1,2,4)

for frame in frame_interval:
  for i in range(1, input_frames_length+1-frame):
    del_1, del_2 = optical_flow(img_convert(load_image('../input_content_frames/frame_%s.jpg' %str(i))), img_convert(load_image('../input_content_frames/frame_%s.jpg' %str(i+frame))), str(i), str(i+frame))
    print('Forward flow from %d to %d completed!' %(i, i+frame))
  for i in range(input_frames_length, frame, -1):
    del_1, del_2 = optical_flow(img_convert(load_image('../input_content_frames/frame_%s.jpg' %str(i))), img_convert(load_image('../input_content_frames/frame_%s.jpg' %str(i-frame))), str(i), str(i-frame))
    print('Backward flow from %d to %d completed!' %(i, i-frame))

In [0]:
def warped_flow_and_pixel_weight(Image1_index, Image2_index):
  backward_flow = np.load('../flow/flow_matrix/outFlow_%s_%s.npy' %(Image2_index, Image1_index))
  h, w, _ = backward_flow.shape
  coordinate = np.rint(backward_flow + 
                       np.concatenate((np.repeat(np.arange(h)+1, w).reshape(h,w,1), 
                                       np.tile(np.arange(w)+1, h).reshape(h,w,1)), axis=2))
  flow = []
  forward_flow = np.load('../flow/flow_matrix/outFlow_%s_%s.npy' %(Image1_index, Image2_index))
  for i in range(coordinate.shape[0]):
    for j in range(coordinate.shape[1]):
      x, y = coordinate[i,j]
      flow.append(forward_flow[min(h-1,max(0,int(x))),min(w-1,max(0,int(y)))])
  warped_flow = np.array(flow).reshape(h,w,-1)
  np.save('../flow/flow_matrix/outFlow_%s_%s_warped.npy' %(Image1_index, Image2_index), warped_flow)
  disocclusion = np.square(np.linalg.norm(warped_flow + backward_flow, axis=2)) - 0.01*(np.square(np.linalg.norm(warped_flow, axis=2))+np.square(np.linalg.norm(backward_flow, axis=2))) - 0.5 > 0
  motion_boundary = np.square(np.gradient(backward_flow, axis=2)[:,:,0])+np.square(np.gradient(backward_flow, axis=2)[:,:,1]) - 0.01*np.square(np.linalg.norm(backward_flow, axis=2)) - 0.002 > 0
  np.save('../flow/flow_matrix/pixel_weight_%s_%s.npy' %(Image1_index, Image2_index), (~(disocclusion + motion_boundary)).astype(int))

In [0]:
frame_interval = (1,2,4)
for frame in frame_interval:
  for i in range(1, input_frames_length+1-frame):
    warped_flow_and_pixel_weight(i, i+frame)
    print('Warped flow and pixel weigt between %d and %d completed!' %(i, i+frame))

In [0]:
def long_pixel_weight(Image2_index):
  if Image2_index == 2:
    c_iminus1_i = np.load('../flow/flow_matrix/pixel_weight_%s_%s.npy' %(Image2_index-1, Image2_index))
    np.save('../flow/flow_matrix/long_pixel_weight_%s_%s.npy' %(Image2_index-1, Image2_index), c_iminus1_i)
  elif Image2_index == 3:
    c_iminus1_i = np.load('../flow/flow_matrix/pixel_weight_%s_%s.npy' %(Image2_index-1, Image2_index))
    np.save('../flow/flow_matrix/long_pixel_weight_%s_%s.npy' %(Image2_index-1, Image2_index), c_iminus1_i)
    c_iminus2_i = np.load('../flow/flow_matrix/pixel_weight_%s_%s.npy' %(Image2_index-2, Image2_index))
    np.save('../flow/flow_matrix/long_pixel_weight_%s_%s.npy' %(Image2_index-2, Image2_index), np.maximum(c_iminus2_i - c_iminus1_i, np.zeros(c_iminus1_i.shape)))
  elif Image2_index == 4:
    c_iminus1_i = np.load('../flow/flow_matrix/pixel_weight_%s_%s.npy' %(Image2_index-1, Image2_index))
    np.save('../flow/flow_matrix/long_pixel_weight_%s_%s.npy' %(Image2_index-1, Image2_index), c_iminus1_i)
    c_iminus2_i = np.load('../flow/flow_matrix/pixel_weight_%s_%s.npy' %(Image2_index-2, Image2_index))
    np.save('../flow/flow_matrix/long_pixel_weight_%s_%s.npy' %(Image2_index-2, Image2_index), np.maximum(c_iminus2_i - c_iminus1_i, np.zeros(c_iminus1_i.shape)))
  else:
    c_iminus1_i = np.load('../flow/flow_matrix/pixel_weight_%s_%s.npy' %(Image2_index-1, Image2_index))
    np.save('../flow/flow_matrix/long_pixel_weight_%s_%s.npy' %(Image2_index-1, Image2_index), c_iminus1_i)
    c_iminus2_i = np.load('../flow/flow_matrix/pixel_weight_%s_%s.npy' %(Image2_index-2, Image2_index))
    np.save('../flow/flow_matrix/long_pixel_weight_%s_%s.npy' %(Image2_index-2, Image2_index), np.maximum(c_iminus2_i - c_iminus1_i, np.zeros(c_iminus1_i.shape))) 
    c_iminus4_i = np.load('../flow/flow_matrix/pixel_weight_%s_%s.npy' %(Image2_index-4, Image2_index))
    np.save('../flow/flow_matrix/long_pixel_weight_%s_%s.npy' %(Image2_index-4, Image2_index), np.maximum(c_iminus4_i - c_iminus2_i - c_iminus1_i, np.zeros(c_iminus1_i.shape)))

In [0]:
for frame in range(2, input_frames_length+1):
  long_pixel_weight(frame)
  print('Long pixel weights for frame %d completed!' %frame)

In [0]:
def warp_flow(Image1_index, Image2_index):
  flow = np.load('../flow/flow_matrix/outFlow_%s_%s_warped.npy' %(Image1_index, Image2_index))
  Image1 = img_convert(load_image('../output_style_transferred_frames/st_frame_%d.jpg' %Image1_index))
  im1 = np.array(Image1, order='C').astype(float)
  h, w = flow.shape[:2]
  flow = -flow
  flow[:,:,0] += np.arange(w)
  flow[:,:,1] += np.arange(h)[:,np.newaxis]
  flow = 2*flow/(flow.max(axis=(0,1)) - flow.min(axis=(0,1)))-1
  warped_frame = torch.nn.functional.grid_sample(torch.from_numpy(im1)[np.newaxis, :].permute(0, 3, 1, 2), torch.from_numpy(flow)[np.newaxis, :])
  plt.imsave('../input_content_frames/warped/frame_%s_%s_warped.jpg' %(Image1_index, Image2_index), warped_frame.permute(0, 2, 3, 1).squeeze().numpy())
  # return warped_frame.float()

In [0]:
def apply_style_transfer(content_img_dir, style_img_dir, style_img_name, output_img_dir):
    content_image_count = 1
    
    num_content_imgs = len([name for name in os.listdir('../%s' %content_img_dir) if os.path.isfile(os.path.join('../', content_img_dir, name))])
    
    while(content_image_count <= num_content_imgs):
        content_image = load_image('../' + content_img_dir + '/frame_' + str(content_image_count) + '.jpg').to(device)

        style_image = load_image('../' + style_img_dir + '/' + style_img_name, shape=content_image.shape[-2:]).to(device)

        # Retrieve the features:
        content_features = get_features(content_image, styleTransferModel)
        style_features = get_features(style_image, styleTransferModel)

        # Calculate the gram matrix for each of our style representations:
        style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

        if content_image_count == 1:
          target_image = content_image.clone().requires_grad_(True).to(device)
          # Number of iterations to update your image:
          steps = 3000
          show_every = 200
        else:
          warp_flow(content_image_count-1, content_image_count)
          if content_image_count-2 > 0:
            warp_flow(content_image_count-2, content_image_count)
          if content_image_count-4 > 0:
            warp_flow(content_image_count-4, content_image_count)
          target_image = load_image('../' + content_img_dir + '/warped/frame_%d_%d_warped.jpg' %(content_image_count-1, content_image_count)).to(device)
          target_image = target_image.clone().requires_grad_(True).to(device)
          # Number of iterations to update your image:
          steps = 800
          show_every = 100

        style_weights = {'conv1_1': 0.25,
                         'conv2_1': 0.25,
                         'conv3_1': 0.25,
                         'conv4_1': 0.25,
                         'conv5_1': 0} # we do not use conv5_1

        content_weight = 50  # alpha
        style_weight = 150  # beta
        longterm_weight = 750 # gamma

        optimizer = optim.Adam([target_image], lr=0.01)
        
        for ii in range(1, steps+1):
            # Get the features from the target image:
            target_features = get_features(target_image, styleTransferModel)
            
            # 1. Calculate the content loss:
            batch_size, depth, height, width = content_features['conv4_1'].shape
            content_loss = torch.mean((target_features['conv4_1'] - content_features['conv4_1']) ** 2)
            
            # 2. Calculate the style loss:
            style_loss = 0
            for layer in style_weights:
                target_feature = target_features[layer]
                batch_size, depth, height, width = target_feature.shape
                target_gram = gram_matrix(target_feature)
                style_gram = style_grams[layer]
                layer_style_loss = style_weights[layer] * torch.mean((target_gram - style_gram) ** 2)
                style_loss += layer_style_loss / (depth * height * width)

            # 3. Calculate the long-term temporal loss:
            longterm_loss = 0
            if content_image_count == 2:
              batch_size, depth, height, width = load_image('../' + content_img_dir + '/warped/frame_%d_%d_warped.jpg' %(content_image_count-1, content_image_count)).shape
              temporal_loss = torch.sum(((target_image - load_image('../' + content_img_dir + '/warped/frame_%d_%d_warped.jpg' %(content_image_count-1, content_image_count)).to(device))**2) * 
                                        (torch.from_numpy(np.repeat(np.load('../flow/flow_matrix/long_pixel_weight_%d_%d.npy' %(content_image_count-1, content_image_count))[:,:,np.newaxis], 
                                                                    depth, axis=2)).to(device).unsqueeze(0).permute(0,3,1,2))) / (depth * height * width)
              longterm_loss += temporal_loss
            if ((content_image_count == 3) | (content_image_count == 4)):
              batch_size, depth, height, width = load_image('../' + content_img_dir + '/warped/frame_%d_%d_warped.jpg' %(content_image_count-1, content_image_count)).shape
              for frame_gap in (1,2):
                temporal_loss = torch.sum(((target_image - load_image('../' + content_img_dir + '/warped/frame_%d_%d_warped.jpg' %(content_image_count-frame_gap, content_image_count)).to(device))**2) * 
                                        (torch.from_numpy(np.repeat(np.load('../flow/flow_matrix/long_pixel_weight_%d_%d.npy' %(content_image_count-frame_gap, content_image_count))[:,:,np.newaxis], 
                                                                    depth, axis=2)).to(device).unsqueeze(0).permute(0,3,1,2))) / (depth * height * width)
                longterm_loss += temporal_loss
            if content_image_count > 4:
              batch_size, depth, height, width = load_image('../' + content_img_dir + '/warped/frame_%d_%d_warped.jpg' %(content_image_count-1, content_image_count)).shape
              for frame_gap in (1,2,4):
                temporal_loss = torch.sum(((target_image - load_image('../' + content_img_dir + '/warped/frame_%d_%d_warped.jpg' %(content_image_count-frame_gap, content_image_count)).to(device))**2) * 
                                        (torch.from_numpy(np.repeat(np.load('../flow/flow_matrix/long_pixel_weight_%d_%d.npy' %(content_image_count-frame_gap, content_image_count))[:,:,np.newaxis], 
                                                                    depth, axis=2)).to(device).unsqueeze(0).permute(0,3,1,2))) / (depth * height * width)
                longterm_loss += temporal_loss

            # Calculate the total loss:
            total_loss = content_weight * content_loss + style_weight * style_loss + longterm_weight * longterm_loss

            # Update the target image:
            optimizer.zero_grad()  # zero out the gradients from previous iterations
            total_loss.backward()  # Backpropagate the loss
            optimizer.step()  # Update the target image

            # Display the intermediate results if required:
            if ii % show_every == 0:
                print("Frame %s Total loss: " %content_image_count, total_loss.item())

        # Save the style transferred image:
        target_image = img_convert(target_image)
        plt.imsave('../output_style_transferred_frames/st_frame_%d.jpg' % content_image_count, target_image)
        print("completed style transfer on image: ", content_image_count)
        content_image_count += 1

In [0]:
# Apply style transfer on the input frames:
apply_style_transfer('input_content_frames', 'input_style_frames', 'style_3.jpg', 'output_style_transferred_frames')

In [0]:
# Convert the final images to video:
def img_to_video(st_output_dir, output_video_name):
    img = cv2.imread('../' + st_output_dir + '/st_frame_1.jpg')
    height, width, layers = img.shape
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video = cv2.VideoWriter('../output_processed_video/%s.avi' %output_video_name, fourcc, 25, (width, height))
    for i in range(1, len([name for name in os.listdir('../%s' %st_output_dir) if os.path.isfile(os.path.join('../', st_output_dir, name))])+1):
        video.write(cv2.imread('../' + st_output_dir + '/st_frame_' + str(i) + '.jpg'))

    cv2.destroyAllWindows()
    video.release()

In [0]:
img_to_video('output_style_transferred_frames', 'style_transfered_video')

#Evaluation

In [0]:
content_image_count = 1
content_image = load_image('../input_content_frames/frame_1.jpg')
content_img_dir = 'input_content_frames'
num_content_imgs = len([name for name in os.listdir('../%s' %content_img_dir) if os.path.isfile(os.path.join('../', content_img_dir, name))])
longterm_weight = 750 # gamma
content_weight = 50  # alpha
style_weight = 150

tot_loss = []
temp_loss = []

while(content_image_count <= num_content_imgs):
    content_image = load_image('../' + content_img_dir + '/frame_' + str(content_image_count) + '.jpg').to(device)
    style_image = load_image('../' + 'input_style_frames' + '/' + 'style_3.jpg', shape=content_image.shape[-2:]).to(device)
    target_image = load_image('../output_style_transferred_frames/st_frame_%d.jpg' % content_image_count, shape=content_image.shape[-2:]).to(device)
    #target_image = load_image('../output_frames_with_temporal/st_frame_%d.jpg' % content_image_count, shape=content_image.shape[-2:]).to(device)

    content_features = get_features(content_image, styleTransferModel)
    style_features = get_features(style_image, styleTransferModel)
    target_features = get_features(target_image, styleTransferModel)
    
    style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

    style_weights = {'conv1_1': 0.25,'conv2_1': 0.25,'conv3_1': 0.25,'conv4_1': 0.25,'conv5_1': 0} 

    # 1. Calculate the content loss:
    batch_size, depth, height, width = content_features['conv4_1'].shape
    content_loss = torch.mean((target_features['conv4_1'] - content_features['conv4_1']) ** 2)

    style_loss = torch.tensor(0.)
    # iterate through each style layer and add to the style loss:
    for layer in style_weights:
        # Get the "target" style representation for the layer:
        target_feature = target_features[layer]
        batch_size, depth, height, width = target_feature.shape

        # Calculate the target gram matrix:
        target_gram = gram_matrix(target_feature)

        # Get the "style" from the style gram matrices computed earlier:
        style_gram = style_grams[layer]

        # the style loss for one layer, weighted appropriately:
        layer_style_loss = style_weights[layer] * torch.mean((target_gram - style_gram) ** 2)

        # Add to the style loss:
        style_loss += layer_style_loss / (depth * height * width)

    # 3. Calculate the long-term temporal loss:
    longterm_loss = torch.tensor(0.)
    if content_image_count == 2:
      batch_size, depth, height, width = load_image('../' + content_img_dir + '/warped/frame_%d_%d_warped.jpg' %(content_image_count-1, content_image_count)).shape
      temporal_loss = torch.sum(((target_image - load_image('../' + content_img_dir + '/warped/frame_%d_%d_warped.jpg' %(content_image_count-1, content_image_count)).to(device))**2) * 
                                (torch.from_numpy(np.repeat(np.load('../flow/flow_matrix/long_pixel_weight_%d_%d.npy' %(content_image_count-1, content_image_count))[:,:,np.newaxis], 
                                                            depth, axis=2)).to(device).unsqueeze(0).permute(0,3,1,2))) / (depth * height * width)
      longterm_loss += temporal_loss
    if ((content_image_count == 3) | (content_image_count == 4)):
      batch_size, depth, height, width = load_image('../' + content_img_dir + '/warped/frame_%d_%d_warped.jpg' %(content_image_count-1, content_image_count)).shape
      for frame_gap in (1,2):
        temporal_loss = torch.sum(((target_image - load_image('../' + content_img_dir + '/warped/frame_%d_%d_warped.jpg' %(content_image_count-frame_gap, content_image_count)).to(device))**2) * 
                                (torch.from_numpy(np.repeat(np.load('../flow/flow_matrix/long_pixel_weight_%d_%d.npy' %(content_image_count-frame_gap, content_image_count))[:,:,np.newaxis], 
                                                            depth, axis=2)).to(device).unsqueeze(0).permute(0,3,1,2))) / (depth * height * width)
        longterm_loss += temporal_loss
    if content_image_count > 4:
      batch_size, depth, height, width = load_image('../' + content_img_dir + '/warped/frame_%d_%d_warped.jpg' %(content_image_count-1, content_image_count)).shape
      for frame_gap in (1,2,4):
        temporal_loss = torch.sum(((target_image - load_image('../' + content_img_dir + '/warped/frame_%d_%d_warped.jpg' %(content_image_count-frame_gap, content_image_count)).to(device))**2) * 
                                (torch.from_numpy(np.repeat(np.load('../flow/flow_matrix/long_pixel_weight_%d_%d.npy' %(content_image_count-frame_gap, content_image_count))[:,:,np.newaxis], 
                                                            depth, axis=2)).to(device).unsqueeze(0).permute(0,3,1,2))) / (depth * height * width)
        longterm_loss += temporal_loss

    tot_loss.append((longterm_loss*longterm_weight+content_loss*content_weight+style_loss*style_weight).item()) 
    temp_loss.append(longterm_loss.item()*longterm_weight)   

    print("completed style transfer on image: ", content_image_count)
    content_image_count += 1

In [0]:
total_loss_dep = tot_loss
temp_loss_dep = temp_loss

In [0]:
temp_loss_ind = temp_loss
total_loss_ind = tot_loss

In [0]:
plt.plot(temp_loss_dep[:210], label='dependency temp loss')
plt.plot(total_loss_dep[:210], label='dependency total loss')
plt.plot(total_loss_ind[:210], label='independent total loss')
plt.plot(temp_loss_ind[:210], label='independent temp loss')
plt.xlabel('Frame')
plt.ylabel('Loss')
plt.title('Total/temporal loss comparison')
plt.legend()
plt.show()

In [0]:
plt.plot(np.array(total_loss_dep[:210])-np.array(temp_loss_dep[:210]), label='dependency')
plt.plot(np.array(total_loss_ind[:210])-np.array(temp_loss_ind[:210]), label='independent')
plt.xlabel('Frame')
plt.ylabel('Loss')
plt.title('Content+Style loss comparison')
plt.legend()
plt.show()

In [0]:
plt.plot(temp_loss_dep[:210], label='dependency')
plt.plot(temp_loss_ind[:210], label='independent')
plt.xlabel('Frame')
plt.ylabel('Loss')
plt.title('Temporal loss comparison')
plt.legend()
plt.show()